In [1]:
!pip install fancyimpute
!pip install imbalanced-learn

In [2]:
import pandas as pd
from fancyimpute import IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# show all columns
pd.options.display.max_columns = None

In [4]:
df_train = pd.read_csv('new_epl_training.csv')
df_train

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum (mm),snowfall_sum (cm),wind_speed_10m_max (km/h),Season,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,Fouls_Difference,Yellows_Difference,Reds_Difference,Referee_Bias_Index,HomeTeam_Elo_Before_Match,AwayTeam_Elo_Before_Match,HomeTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio,EPL,HomeTeam_CupGamesLastMonth,AwayTeam_CupGamesLastMonth,HomeTeam_Morale,AwayTeam_Morale,Curse_Parameter
0,2000-08-19,Charlton,Man City,4,0.0,H,2,0,H,Rob Harris,17,8,14,4,6,6,13,12,1,2,0,0,20.5,13.7,16.9,2.6,0.0,16.0,2000-2001,173.216250,307.286550,2073.122369,774.473532,1686.839900,584.415044,1679.712000,1970.531638,1,-1,0,0.000000,1500.000000,1500.000000,0.235294,0.000000,0.285714,0.000000,0.134048,0.108753,0.230415,0.235632,0.121277,0.139785,0.257919,0.292793,0.578947,0.210526,0.620000,0.512195,0.785714,0.500000,1,0,0,0,0,0
1,2000-08-19,Chelsea,West Ham,4,2.0,H,1,0,H,Graham Barber,17,12,10,5,7,7,19,14,1,2,0,0,20.5,13.7,16.9,2.6,0.0,16.0,2000-2001,136.056680,135.192982,1691.299854,1242.118239,1704.834412,895.031110,901.836801,832.980702,5,-1,0,1.333333,1500.000000,1500.000000,0.235294,0.166667,0.400000,0.400000,0.134921,0.094737,0.283333,0.196507,0.122616,0.105708,0.252809,0.228311,0.684211,0.210526,0.647059,0.466667,0.764706,0.400000,1,0,0,0,0,0
2,2000-08-19,Coventry,Middlesbrough,1,3.0,A,1,1,D,Barry Knight,6,16,3,9,8,4,15,21,5,3,1,0,18.0,11.4,14.4,0.9,0.0,17.7,2000-2001,191.449539,133.710790,1492.648105,1824.347684,1095.830300,1366.709700,1856.523790,1296.619790,-6,2,1,-1.000000,1500.000000,1500.000000,0.166667,0.187500,0.333333,0.333333,0.092545,0.125000,0.203390,0.263473,0.130435,0.093418,0.253012,0.196429,0.210526,0.263158,0.388889,0.590909,0.500000,0.555556,1,0,0,0,0,0
3,2000-08-19,Derby,Southampton,2,2.0,D,1,2,A,Andy D'Urso,6,13,4,6,5,8,11,13,1,1,0,0,17.7,9.3,13.7,2.8,0.0,18.6,2000-2001,179.294013,145.866316,1534.110553,1658.497895,1366.709700,1686.839900,1738.649263,1414.494316,-2,0,0,-0.666667,1500.000000,1500.000000,0.333333,0.153846,0.500000,0.333333,0.121311,0.096618,0.262411,0.229885,0.123950,0.118227,0.232283,0.238806,0.421053,0.157895,0.621622,0.325000,0.800000,0.214286,1,0,0,0,0,0
4,2000-08-19,Leeds,Everton,2,0.0,H,2,0,H,Dermot Gallagher,17,12,8,6,6,4,21,20,1,3,0,0,18.0,10.1,13.6,4.0,0.0,20.8,2000-2001,130.671908,237.180000,2653.596631,1448.825210,2203.973300,1548.376343,1267.151157,1459.607832,1,-2,0,-0.333333,1500.000000,1500.000000,0.117647,0.000000,0.250000,0.000000,0.134737,0.114213,0.280702,0.230769,0.111399,0.153247,0.218274,0.308901,0.578947,0.263158,0.562500,0.355556,0.550000,0.454545,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11887,2023-10-29,Brighton,Fulham,1,1.0,D,1,0,H,M Salisbury,18,10,7,5,7,3,12,8,0,3,0,0,12.9,10.7,11.8,12.0,0.0,34.4,2023-2024,179.050000,192.300000,1617.212000,1547.346500,1269.134250,1215.230500,1274.491000,1193.6530

In [5]:
df_test = pd.read_csv('epl-test.csv')
df_test

,Date,HomeTeam,AwayTeam
0,03-Feb-24,AFC Bournemouth,Nottingham Forest
1,03-Feb-24,Arsenal,Liverpool
2,03-Feb-24,Brentford,Man City
3,03-Feb-24,Brighton,Crystal Palace
4,03-Feb-24,Burnley,Fulham
5,03-Feb-24,Chelsea,Wolves
6,03-Feb-24,Everton,Spurs
7,03-Feb-24,Man Utd,West Ham
8,03-Feb-24,Newcastle,Luton Town
9,03-Feb-24,Sheff Utd,Aston Villa


In [6]:
# make test file consistent with train file
team_names_changes = {
    'AFC Bournemouth': 'Bournemouth',
    'Nottingham Forest': "Nott'm Forest",
    'Spurs': 'Tottenham',
    'Man Utd': 'Man United',
    'Luton Town': 'Luton',
    'Sheff Utd': 'Sheffield United'
}

df_test['HomeTeam'] = df_test['HomeTeam'].replace(team_names_changes)
df_test['AwayTeam'] = df_test['AwayTeam'].replace(team_names_changes)

df_test

,Date,HomeTeam,AwayTeam
0,03-Feb-24,Bournemouth,Nott'm Forest
1,03-Feb-24,Arsenal,Liverpool
2,03-Feb-24,Brentford,Man City
3,03-Feb-24,Brighton,Crystal Palace
4,03-Feb-24,Burnley,Fulham
5,03-Feb-24,Chelsea,Wolves
6,03-Feb-24,Everton,Tottenham
7,03-Feb-24,Man United,West Ham
8,03-Feb-24,Newcastle,Luton
9,03-Feb-24,Sheffield United,Aston Villa


In [7]:
# add data we already know
df_test['Season'] = '2023-2024'
df_test['EPL'] = 1

def get_ability(team, ability_name, isHome):
    if isHome:
      filtered_df = df_train[(df_train['Season'] == '2023-2024') & ((df_train['HomeTeam'] == team))]
    else:
      filtered_df = df_train[(df_train['Season'] == '2023-2024') & ((df_train['AwayTeam'] == team))]

    ability_col = filtered_df[ability_name]

    try:
        ability = ability_col.iloc[0]
        return ability
    except ValueError:
        print("No matching rows found")


# add abilities based on seasons as they are consistent throughout the season
df_test['Home_GK_Ability'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'Home_GK_Ability', True))
df_test['Away_GK_Ability'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'Away_GK_Ability', False))
df_test['Home_team_offensive_ability'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'Home_team_offensive_ability', True))
df_test['Away_team_offensive_ability'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'Away_team_offensive_ability', False))
df_test['Home_team_possession_ability'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'Home_team_possession_ability', True))
df_test['Away_team_possession_ability'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'Away_team_possession_ability', False))
df_test['Home_team_defensive_ability'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'Home_team_defensive_ability', True))
df_test['Away_team_defensive_ability'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'Away_team_defensive_ability', False))
df_test['HomeTeam_Season_Offensive_Shooting_Efficiency'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'HomeTeam_Season_Offensive_Shooting_Efficiency', True))
df_test['AwayTeam_Season_Offensive_Shooting_Efficiency'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'AwayTeam_Season_Offensive_Shooting_Efficiency', False))
df_test['HomeTeam_Season_Offensive_Shots_on_Target_Efficiency'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'HomeTeam_Season_Offensive_Shots_on_Target_Efficiency', True))
df_test['AwayTeam_Season_Offensive_Shots_on_Target_Efficiency'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'AwayTeam_Season_Offensive_Shots_on_Target_Efficiency', False))
df_test['HomeTeam_Season_Defensive_Shooting_Efficiency'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'HomeTeam_Season_Defensive_Shooting_Efficiency', True))
df_test['AwayTeam_Season_Defensive_Shooting_Efficiency'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'AwayTeam_Season_Defensive_Shooting_Efficiency', False))
df_test['HomeTeam_Season_Defensive_Shots_on_Target_Efficiency'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'HomeTeam_Season_Defensive_Shots_on_Target_Efficiency', True))
df_test['AwayTeam_Season_Defensive_Shots_on_Target_Efficiency'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'AwayTeam_Season_Defensive_Shots_on_Target_Efficiency', False))
df_test['HomeTeam_HomeWinRate'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'HomeTeam_HomeWinRate', True))
df_test['AwayTeam_AwayWinRate'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'AwayTeam_AwayWinRate', False))
df_test['HomeTeam_HomeGoalsRatio'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'HomeTeam_HomeGoalsRatio', True))
df_test['AwayTeam_AwayGoalsRatio'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'AwayTeam_AwayGoalsRatio', False))
df_test['HomeTeam_HomeWinsRatio'] = df_test['HomeTeam'].apply(lambda team: get_ability(team, 'HomeTeam_HomeWinsRatio', True))
df_test['AwayTeam_AwayWinsRatio'] = df_test['AwayTeam'].apply(lambda team: get_ability(team, 'AwayTeam_AwayWinsRatio', False))




df_test


,Date,HomeTeam,AwayTeam,Season,EPL,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio
0,03-Feb-24,Bournemouth,Nott'm Forest,2023-2024,1,187.9500,189.1000,1484.69200,1450.72700,1204.88375,1120.69550,1147.7150,1265.48300,0.070175,0.089286,0.195122,0.285714,0.121387,0.108696,0.338710,0.319149,0.166667,0.00,0.500000,0.400000,1.000000,0.500000
1,03-Feb-24,Arsenal,Liverpool,2023-2024,1,204.0000,205.3500,1749.07250,1784.21700,1349.41250,1385.04050,1316.3200,1295.19600,0.159722,0.130682,0.403509,0.425926,0.095238,0.069231,0.363636,0.230769,0.666667,0.75,0.652174,0.391304,0.571429,0.285714
2,03-Feb-24,Brentford,Man City,2023-2024,1,187.6500,202.1000,1589.68650,1704.78625,1175.31350,1352.69100,1157.1270,1296.00350,0.111888,0.135802,0.326531,0.314286,0.090909,0.102941,0.315789,0.304348,0.200000,0.40,0.562500,0.545455,0.333333,0.500000
3,03-Feb-24,Brighton,Crystal Palace,2023-2024,1,179.0500,187.9500,1617.21200,1485.11500,1269.13425,1153.35000,1274.4910,1182.31650,0.149351,0.062500,0.359375,0.190476,0.152000,0.102362,0.422222,0.361111,0.500000,0.50,0.608696,0.500000,0.600000,0.666667
4,03-Feb-24,Burnley,Fulham,2023-2024,1,115.4775,192.3000,1575.57300,1547.34650,1231.27000,1215.23050,1119.8080,1193.65300,0.082474,0.084906,0.296296,0.257143,0.155280,0.128000,0.390625,0.285714,0.000000,0.20,0.500000,0.555556,0.000000,0.333333
5,03-Feb-24,Chelsea,Wolves,2023-2024,1,186.7000,128.7275,1535.84950,1502.80300,1260.09500,1147.40750,1237.5205,1147.52200,0.093525,0.110169,0.295455,0.333333,0.099099,0.106918,0.305556,0.283333,0.166667,0.50,0.461538,0.461538,0.333333,0.666667
6,03-Feb-24,Everton,Tottenham,2023-2024,1,190.9500,195.8000,1529.31550,1590.96625,1129.89625,1287.77075,1175.1080,1254.50250,0.067114,0.124294,0.204082,0.360656,0.108527,0.070866,0.358974,0.214286,0.200000,0.40,0.400000,0.636364,0.333333,0.500000
7,03-Feb-24,Man United,West Ham,2023-2024,1,193.0000,192.6500,1570.99325,1704.46225,1284.86325,1209.50150,1251.0430,1186.99750,0.072848,0.136752,0.229167,0.400000,0.114286,0.097701,0.296296,0.293103,0.500000,0.50,0.636364,0.500000,0.600000,0.500000
8,03-Feb-24,Newcastle,Luton,2023-2024,1,196.1500,182.6500,1712.48550,1592.92000,1274.44000,1070.51300,1223.7830,1057.81875,0.185714,0.073770,0.406250,0.391304,0.095652,0.125786,0.297297,0.363636,0.800000,0.20,0.500000,0.666667,0.800000,1.000000
9,03-Feb-24,Sheffield United,Aston Villa,2023-2024,1,127.8275,194.1000,1414.66850,1568.45650,1058.32350,1280.52000,1129.5825,1252.10500,0.082353,0.168831,0.233333,0.433333,0.142857,0.129630,0.362500,0.358974,0.000000,0.00,0.571429,0.230769,0.000000,0.285714


In [8]:
df_combined = pd.concat([df_train, df_test], ignore_index=True)
df_combined

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum (mm),snowfall_sum (cm),wind_speed_10m_max (km/h),Season,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,Fouls_Difference,Yellows_Difference,Reds_Difference,Referee_Bias_Index,HomeTeam_Elo_Before_Match,AwayTeam_Elo_Before_Match,HomeTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio,EPL,HomeTeam_CupGamesLastMonth,AwayTeam_CupGamesLastMonth,HomeTeam_Morale,AwayTeam_Morale,Curse_Parameter
0,2000-08-19,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,Rob Harris,17.0,8.0,14.0,4.0,6.0,6.0,13.0,12.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,2000-2001,173.216250,307.286550,2073.122369,774.473532,1686.839900,584.415044,1679.712000,1970.531638,1.0,-1.0,0.0,0.000000,1500.0,1500.0,0.235294,0.000000,0.285714,0.000000,0.134048,0.108753,0.230415,0.235632,0.121277,0.139785,0.257919,0.292793,0.578947,0.210526,0.620000,0.512195,0.785714,0.500000,1,0.0,0.0,0.0,0.0,0.0
1,2000-08-19,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,Graham Barber,17.0,12.0,10.0,5.0,7.0,7.0,19.0,14.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,2000-2001,136.056680,135.192982,1691.299854,1242.118239,1704.834412,895.031110,901.836801,832.980702,5.0,-1.0,0.0,1.333333,1500.0,1500.0,0.235294,0.166667,0.400000,0.400000,0.134921,0.094737,0.283333,0.196507,0.122616,0.105708,0.252809,0.228311,0.684211,0.210526,0.647059,0.466667,0.764706,0.400000,1,0.0,0.0,0.0,0.0,0.0
2,2000-08-19,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,Barry Knight,6.0,16.0,3.0,9.0,8.0,4.0,15.0,21.0,5.0,3.0,1.0,0.0,18.0,11.4,14.4,0.9,0.0,17.7,2000-2001,191.449539,133.710790,1492.648105,1824.347684,1095.830300,1366.709700,1856.523790,1296.619790,-6.0,2.0,1.0,-1.000000,1500.0,1500.0,0.166667,0.187500,0.333333,0.333333,0.092545,0.125000,0.203390,0.263473,0.130435,0.093418,0.253012,0.196429,0.210526,0.263158,0.388889,0.590909,0.500000,0.555556,1,0.0,0.0,0.0,0.0,0.0
3,2000-08-19,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,Andy D'Urso,6.0,13.0,4.0,6.0,5.0,8.0,11.0,13.0,1.0,1.0,0.0,0.0,17.7,9.3,13.7,2.8,0.0,18.6,2000-2001,179.294013,145.866316,1534.110553,1658.497895,1366.709700,1686.839900,1738.649263,1414.494316,-2.0,0.0,0.0,-0.666667,1500.0,1500.0,0.333333,0.153846,0.500000,0.333333,0.121311,0.096618,0.262411,0.229885,0.123950,0.118227,0.232283,0.238806,0.421053,0.157895,0.621622,0.325000,0.800000,0.214286,1,0.0,0.0,0.0,0.0,0.0
4,2000-08-19,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,Dermot Gallagher,17.0,12.0,8.0,6.0,6.0,4.0,21.0,20.0,1.0,3.0,0.0,0.0,18.0,10.1,13.6,4.0,0.0,20.8,2000-2001,130.671908,237.180000,2653.596631,1448.825210,2203.973300,1548.376343,1267.151157,1459.607832,1.0,-2.0,0.0,-0.333333,1500.0,1500.0,0.117647,0.000000,0.250000,0.000000,0.134737,0.114213,0.280702,0.230769,0.111399,0.153247,0.218274,0.308901,0.578947,0.263158,0.562500,0.355556,0.550000,0.454545,1,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,03-Feb-24,Chelsea,Wolves,NaN,N

In [9]:
df_combined.drop(['Referee', 'FTHG', 'FTAG', 'HTR', 'HTHG', 'HTAG','HST','AST','HS','AS'], axis=1, inplace=True)

In [10]:
# drop all the cup games
df_combined = df_combined[df_combined['EPL'] != 0]
df_combined.drop('EPL', axis=1, inplace=True)

<ipython-input-10-d2c87648dea9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined.drop('EPL', axis=1, inplace=True)


In [11]:
df_combined['Date'] = pd.to_datetime(df_combined['Date'])
df_combined['Year'] = df_combined['Date'].dt.year
df_combined['Month'] = df_combined['Date'].dt.month
df_combined['Day'] = df_combined['Date'].dt.day
df_combined.drop('Date', axis=1, inplace=True)

<ipython-input-11-3f60f9cea604>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['Date'] = pd.to_datetime(df_combined['Date'])
<ipython-input-11-3f60f9cea604>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['Year'] = df_combined['Date'].dt.year
<ipython-input-11-3f60f9cea604>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [12]:
# change season to sequential numbers
seasons = sorted(df_combined['Season'].unique())
sequential_season = {season: i+1 for i, season in enumerate(seasons)}
df_combined['Season'] = df_combined['Season'].map(sequential_season)
df_combined

<ipython-input-12-e40c5de34614>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['Season'] = df_combined['Season'].map(sequential_season)


,HomeTeam,AwayTeam,FTR,HC,AC,HF,AF,HY,AY,HR,AR,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum (mm),snowfall_sum (cm),wind_speed_10m_max (km/h),Season,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,Fouls_Difference,Yellows_Difference,Reds_Difference,Referee_Bias_Index,HomeTeam_Elo_Before_Match,AwayTeam_Elo_Before_Match,HomeTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio,HomeTeam_CupGamesLastMonth,AwayTeam_CupGamesLastMonth,HomeTeam_Morale,AwayTeam_Morale,Curse_Parameter,Year,Month,Day
0,Charlton,Man City,H,6.0,6.0,13.0,12.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,1,173.216250,307.286550,2073.122369,774.473532,1686.839900,584.415044,1679.712000,1970.531638,1.0,-1.0,0.0,0.000000,1500.0,1500.0,0.235294,0.000000,0.285714,0.000000,0.134048,0.108753,0.230415,0.235632,0.121277,0.139785,0.257919,0.292793,0.578947,0.210526,0.620000,0.512195,0.785714,0.500000,0.0,0.0,0.0,0.0,0.0,2000,8,19
1,Chelsea,West Ham,H,7.0,7.0,19.0,14.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,1,136.056680,135.192982,1691.299854,1242.118239,1704.834412,895.031110,901.836801,832.980702,5.0,-1.0,0.0,1.333333,1500.0,1500.0,0.235294,0.166667,0.400000,0.400000,0.134921,0.094737,0.283333,0.196507,0.122616,0.105708,0.252809,0.228311,0.684211,0.210526,0.647059,0.466667,0.764706,0.400000,0.0,0.0,0.0,0.0,0.0,2000,8,19
2,Coventry,Middlesbrough,A,8.0,4.0,15.0,21.0,5.0,3.0,1.0,0.0,18.0,11.4,14.4,0.9,0.0,17.7,1,191.449539,133.710790,1492.648105,1824.347684,1095.830300,1366.709700,1856.523790,1296.619790,-6.0,2.0,1.0,-1.000000,1500.0,1500.0,0.166667,0.187500,0.333333,0.333333,0.092545,0.125000,0.203390,0.263473,0.130435,0.093418,0.253012,0.196429,0.210526,0.263158,0.388889,0.590909,0.500000,0.555556,0.0,0.0,0.0,0.0,0.0,2000,8,19
3,Derby,Southampton,D,5.0,8.0,11.0,13.0,1.0,1.0,0.0,0.0,17.7,9.3,13.7,2.8,0.0,18.6,1,179.294013,145.866316,1534.110553,1658.497895,1366.709700,1686.839900,1738.649263,1414.494316,-2.0,0.0,0.0,-0.666667,1500.0,1500.0,0.333333,0.153846,0.500000,0.333333,0.121311,0.096618,0.262411,0.229885,0.123950,0.118227,0.232283,0.238806,0.421053,0.157895,0.621622,0.325000,0.800000,0.214286,0.0,0.0,0.0,0.0,0.0,2000,8,19
4,Leeds,Everton,H,6.0,4.0,21.0,20.0,1.0,3.0,0.0,0.0,18.0,10.1,13.6,4.0,0.0,20.8,1,130.671908,237.180000,2653.596631,1448.825210,2203.973300,1548.376343,1267.151157,1459.607832,1.0,-2.0,0.0,-0.333333,1500.0,1500.0,0.117647,0.000000,0.250000,0.000000,0.134737,0.114213,0.280702,0.230769,0.111399,0.153247,0.218274,0.308901,0.578947,0.263158,0.562500,0.355556,0.550000,0.454545,0.0,0.0,0.0,0.0,0.0,2000,8,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,Chelsea,Wolves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,186.700000,128.727500,1535.849500,1502.803000,1260.095000,1147.407500,1237.520500,1147.522000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.093525,0.110169,0.295455,0.333333,0.099099,0.106918,0.305556,0.283333,0.166667,0.500000,0.461538,0.461538,0.333333,0.666667,NaN,NaN,NaN,NaN,NaN,2024,2,3
11898,Everton,Tottenham,NaN,NaN,NaN,NaN,

In [13]:
X = df_combined.drop('FTR', axis=1)  # Features

In [14]:
label_enc = LabelEncoder()
df_combined['FTR'] = label_enc.fit_transform(df_combined['FTR'])
y = df_combined.iloc[:-10]['FTR']
y

<ipython-input-14-edd6b4392405>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['FTR'] = label_enc.fit_transform(df_combined['FTR'])


0        2
1        2
2        0
3        1
4        2
        ..
11887    1
11888    2
11889    0
11890    2
11891    0
Name: FTR, Length: 8840, dtype: int64

In [15]:
categorical_features = ['HomeTeam', 'AwayTeam']
ohe = OneHotEncoder(sparse=False)
ohe_encoded = ohe.fit_transform(X[categorical_features])
ohe_encoded_columns = ohe.get_feature_names_out(categorical_features)

encoded_df = pd.DataFrame(ohe_encoded, columns=ohe_encoded_columns)
X_processed_df = pd.concat([X.reset_index(drop=True), encoded_df], axis=1)

X_processed_df = X_processed_df.drop(categorical_features, axis=1)
X_processed_df

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,HC,AC,HF,AF,HY,AY,HR,AR,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum (mm),snowfall_sum (cm),wind_speed_10m_max (km/h),Season,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,Fouls_Difference,Yellows_Difference,Reds_Difference,Referee_Bias_Index,HomeTeam_Elo_Before_Match,AwayTeam_Elo_Before_Match,HomeTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio,HomeTeam_CupGamesLastMonth,AwayTeam_CupGamesLastMonth,HomeTeam_Morale,AwayTeam_Morale,Curse_Parameter,Year,Month,Day,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Coventry,HomeTeam_Crystal Palace,HomeTeam_Derby,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Middlesbrough,HomeTeam_Newcastle,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Portsmouth,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Sheffield United,HomeTeam_Southampton,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Swansea,HomeTeam_Tottenham,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Cardiff,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Coventry,AwayTeam_Crystal Palace,AwayTeam_Derby,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Middlesbrough,AwayTeam_Newcastle,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Portsmouth,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Sheffield United,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves
0,6.0,6.0,13.0,12.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,1,173.216250,307.286550,2073.122369,774.473532,1686.839900,584.415044,1679.712000,1970.531638,1.0,-1.0,0.0,0.000000,1500.0,1500.0,0.235294,0.000000,0.285714,0.000000,0.134048,0.108753,0.230415,0.235632,0.121277,0.139785,0.257919,0.292793,0.578947,0.210526,0.620000,0.512195,0.785714,0.500000,0.0,0.0,0.0,0.0,0.0,2000,8,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.0,7.0,19.0,14.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,1,136.056680,135.192982,1691.299854,1242.118239,1704.834412,895.031110,901.836801,832.980702,5.0,-1.0,0.0,1.333333,1500.0,1500.0,0.235294,0.166667,0.400000,0.400000,0.134921,0.094737,0.283333,0.196507,0.12261

In [16]:
original_column_order = X_processed_df.columns

In [17]:
mice_imputer = IterativeImputer()
X_processed_df_all_columns = mice_imputer.fit_transform(X_processed_df)

In [18]:
X_imputed_df = pd.DataFrame(X_processed_df_all_columns, columns=original_column_order)
X_imputed_df

,HC,AC,HF,AF,HY,AY,HR,AR,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum (mm),snowfall_sum (cm),wind_speed_10m_max (km/h),Season,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,Fouls_Difference,Yellows_Difference,Reds_Difference,Referee_Bias_Index,HomeTeam_Elo_Before_Match,AwayTeam_Elo_Before_Match,HomeTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio,HomeTeam_CupGamesLastMonth,AwayTeam_CupGamesLastMonth,HomeTeam_Morale,AwayTeam_Morale,Curse_Parameter,Year,Month,Day,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Coventry,HomeTeam_Crystal Palace,HomeTeam_Derby,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Middlesbrough,HomeTeam_Newcastle,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Portsmouth,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Sheffield United,HomeTeam_Southampton,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Swansea,HomeTeam_Tottenham,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Cardiff,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Coventry,AwayTeam_Crystal Palace,AwayTeam_Derby,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Middlesbrough,AwayTeam_Newcastle,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Portsmouth,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Sheffield United,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves
0,6.000000,6.000000,13.000000,12.000000,1.000000,2.000000,0.000000,0.000000,20.500000,13.700000,16.900000,2.600000,0.000000,16.000000,1.0,173.216250,307.286550,2073.122369,774.473532,1686.839900,584.415044,1679.712000,1970.531638,1.000000,-1.000000,0.000000,0.000000,1500.000000,1500.000000,0.235294,0.000000,0.285714,0.000000,0.134048,0.108753,0.230415,0.235632,0.121277,0.139785,0.257919,0.292793,0.578947,0.210526,0.620000,0.512195,0.785714,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,2000.0,8.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.000000,7.000000,19.000000,14.000000,1.000000,2.000000,0.000000,0.000000,20.500000,13.700000,16.900000,2.600000,0.000000,16.000000,1.0,136.05

In [19]:
numerical_features = list(X_imputed_df.columns.difference(ohe_encoded_columns))
# numerical_features.remove('Year')
# numerical_features.remove('Month')
# numerical_features.remove('Day')
scaler = StandardScaler()
X_imputed_df_scaled = scaler.fit_transform(X_imputed_df[numerical_features])
X_imputed_df_scaled = pd.DataFrame(X_imputed_df_scaled, columns=numerical_features, index=X_imputed_df.index)
X_imputed_df = X_imputed_df.drop(columns=numerical_features)
X_imputed_df = pd.concat([X_imputed_df, X_imputed_df_scaled], axis=1)
X_imputed_df

,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Coventry,HomeTeam_Crystal Palace,HomeTeam_Derby,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Middlesbrough,HomeTeam_Newcastle,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Portsmouth,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Sheffield United,HomeTeam_Southampton,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Swansea,HomeTeam_Tottenham,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Cardiff,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Coventry,AwayTeam_Crystal Palace,AwayTeam_Derby,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Middlesbrough,AwayTeam_Newcastle,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Portsmouth,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Sheffield United,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves,AC,AF,AR,AY,AwayTeam_AwayGoalsRatio,AwayTeam_AwayWinRate,AwayTeam_AwayWinsRatio,AwayTeam_CupGamesLastMonth,AwayTeam_Elo_Before_Match,AwayTeam_Morale,AwayTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,Away_GK_Ability,Away_team_defensive_ability,Away_team_offensive_ability,Away_team_possession_ability,Curse_Parameter,Day,Fouls_Difference,HC,HF,HR,HY,HomeTeam_CupGamesLastMonth,HomeTeam_Elo_Before_Match,HomeTeam_HomeGoalsRatio,HomeTeam_HomeWinRate,HomeTeam_HomeWinsRatio,HomeTeam_Morale,HomeTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,Home_GK_Ability,Home_team_defensive_ability,Home_team_offensive_ability,Home_team_possession_ability,Month,Reds_Difference,Referee_Bias_Index,Season,Year,Yellows_Difference,rain_sum (mm),snowfall_sum (cm),temperature_2m_max,temperature_2m_mean,temperature_2m_min,wind_speed_10m_max (km/h)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.451057,4.816711e-02,-2.962163e-01,1.863578e-01,1.063121,-0.482592,1.011091,-0.575891,-0.570230,-0.013971,-0.951402,-0.970933,1.468412,0.505888,-0.051596,-0.325952,2.861378,1.493209,-1.669031,-1.645730,0.036752,0.332928,3.075503e-01,-0.019301,4.450919e-01,-2.472752e-01,-3.625857e-01,-0.576005,-0.546865,0.681137,0.648487,1.295068,-0.048681,1.086069,0.049485,0.526133,-0.022831,1.104596,-0.403394,-0.029409,0.799764,1.064861,0.769518,0.327866,6.472061e-02,1.500143e-01,-1.657957,-1.723298,-4.261924e-01,0.092787,-0.121938,1.686645,1.723507,1.743266,-0.777942
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [20]:
submission_df = X_imputed_df.tail(10)
submission_df.to_csv('submission_data_for_prediction.csv', index=False)


In [21]:
df = X_imputed_df.iloc[:-10]
df

,Year,Month,Day,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Coventry,HomeTeam_Crystal Palace,HomeTeam_Derby,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Middlesbrough,HomeTeam_Newcastle,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Portsmouth,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Sheffield United,HomeTeam_Southampton,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Swansea,HomeTeam_Tottenham,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Cardiff,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Coventry,AwayTeam_Crystal Palace,AwayTeam_Derby,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Middlesbrough,AwayTeam_Newcastle,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Portsmouth,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Sheffield United,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves,AC,AF,AR,AY,AwayTeam_AwayGoalsRatio,AwayTeam_AwayWinRate,AwayTeam_AwayWinsRatio,AwayTeam_CupGamesLastMonth,AwayTeam_Elo_Before_Match,AwayTeam_Morale,AwayTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,Away_GK_Ability,Away_team_defensive_ability,Away_team_offensive_ability,Away_team_possession_ability,Curse_Parameter,Fouls_Difference,HC,HF,HR,HY,HomeTeam_CupGamesLastMonth,HomeTeam_Elo_Before_Match,HomeTeam_HomeGoalsRatio,HomeTeam_HomeWinRate,HomeTeam_HomeWinsRatio,HomeTeam_Morale,HomeTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,Home_GK_Ability,Home_team_defensive_ability,Home_team_offensive_ability,Home_team_possession_ability,Reds_Difference,Referee_Bias_Index,Season,Yellows_Difference,rain_sum (mm),snowfall_sum (cm),temperature_2m_max,temperature_2m_mean,temperature_2m_min,wind_speed_10m_max (km/h)
0,2000.0,8.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.451057,0.048167,-0.296216,0.186358,1.063121,-0.482592,1.011091,-0.575891,-0.570230,-0.013971,-0.951402,-0.970933,1.468412,0.505888,-0.051596,-0.325952,2.861378,1.493209,-1.669031,-1.645730,0.036752,0.307550,-0.019301,0.445092,-0.247275,-0.362586,-0.576005,-0.546865,0.681137,0.648487,1.295068,-0.048681,1.086069,0.049485,0.526133,-0.022831,1.104596,-0.403394,-0.029409,0.799764,1.064861,0.769518,0.064721,0.150014,-1.657957,-0.426192,0.092787,-0.121938,1.686645,1.723507,1.743266,-0.777942
1,2000.0,8.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

# over sampling
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [23]:
y_train_df = pd.DataFrame(y_train_resampled, columns=['FTR'])
combined_df = pd.concat([X_train_resampled, y_train_df], axis=1)
combined_df.to_csv('final_train_data.csv', index=False)

In [24]:
y_test_df = pd.DataFrame(y_test, columns=['FTR'])
y_test_df.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
combined_df_test = pd.concat([X_test, y_test_df], axis=1)
combined_df_test.to_csv('final_test_data.csv', index=False)